In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


In [ ]:
%load_ext nvcc_plugin

In [4]:
code = """
#include<omp.h>
#include<iostream>
using namespace std;

void swap(float *num1, float *num2) {
    float temp = *num1;
    *num1 = *num2;
    *num2 = temp;
}

int main() {
    int n = 100;
    float a[n];
    
    omp_set_num_threads(2);
    
    for(int i=0; i<n; i++) {
        a[i] = rand()%100;
    }
    
    for(int i=0; i<n; i++) 
        cout<<"  "<<a[i];
    cout<<endl;
    
    int i=0, j=0;
    int first=0;
    double start, end;
    
    start = omp_get_wtime();
    for(i=0; i<n-1; i++) {
        first = i%2;
        #pragma omp parallel for
        for(j=first; j<n-1; j++) {
            if(a[j] > a[j+1])
              swap(&a[j], &a[j+1]);
        }
    }
    
    end = omp_get_wtime();
    cout<<"Result(parallel) : "<<endl;
    for(i=0; i<n; i++)
        for(i=0; i<n; i++)
      cout<<"  "<<a[i];
    cout<<endl;
    
    cout<<"Time parallel = "<<(end-start)<<endl;
    
    return 0;
    
}
"""

In [5]:
text_file = open("code.cpp", "w")
text_file.write(code)
text_file.close()

In [6]:
!g++ -fopenmp code.cpp

In [7]:
!./a.out

  83  86  77  15  93  35  86  92  49  21  62  27  90  59  63  26  40  26  72  36  11  68  67  29  82  30  62  23  67  35  29  2  22  58  69  67  93  56  11  42  29  73  21  19  84  37  98  24  15  70  13  26  91  80  56  73  62  70  96  81  5  25  84  27  36  5  46  29  13  57  24  95  82  45  14  67  34  64  43  50  87  8  76  78  88  84  3  51  54  99  32  60  76  68  39  12  26  86  94  39
Result(parallel) : 
  2  3  5  5  8  11  11  12  13  13  14  15  15  19  21  21  22  23  24  24  25  26  26  26  26  27  27  29  29  29  29  30  32  34  35  35  36  36  37  39  39  40  42  43  45  46  49  50  51  54  56  56  57  58  59  60  62  62  62  63  64  67  67  67  67  68  68  69  70  70  72  73  73  76  76  77  78  80  81  82  82  83  84  84  84  86  86  86  87  88  90  91  92  93  93  94  95  96  98  99
Time parallel = 0.00106859


In [8]:
code = """
#include<iostream>
#include<omp.h>

using namespace std;

void printArray(int *arr, int size) {
    for(int i=0; i<size; i++) {
         cout<<arr[i]<<"  ";
    }
    cout<<endl;
}

void merge(int* arr, int start, int mid, int end) {
    int len = (end - start) + 1;
    int temp[len];
    int cur = 0;
    
    int i = start;
    int j = mid + 1;
    while(i <= mid && j <= end){
        if(arr[i] < arr[j]) {
            temp[cur] = arr[i];
            cur++;
            i++;
        }
        else {
            temp[cur] = arr[j];
            cur++;
            j++;
        }
    }
    if(i <= mid) {
        while(i <= mid) {
            temp[cur] = arr[i];
            i++;
            cur++;
        }
    }
    
    else if(j <= end) {
        while(j <= end) {
            temp[cur] = arr[j];
            j++;
            cur++;
        }
    }
    
    cur = 0;
    for(i=start; i<=end; i++) {
        arr[i] = temp[cur];
        cur++;
    }

}

void mergeSort(int *arr, int start, int end) {
    if(start < end) {
        int mid = (start+end) / 2;
        
        #pragma omp parallel sections
        {
            #pragma omp section
            mergeSort(arr, start, mid);
            
            #pragma omp section
            mergeSort(arr, mid+1, end);
        }
        
        merge(arr, start, mid, end);
    
    }
}

int main(int argc, char *argv[]) {
    int size = 100;
    int a[size];
    
    double start, end;
    
    omp_set_num_threads(2);
    
    for(int i=0; i<size; i++) {
        a[i] = rand()% 100;
    }
    
    //int a[]= {7,33,5,5,23,111,75,34,77,121,120};
    
    for(int i=0; i<size; i++) 
        cout<<"  "<<a[i];
    cout<<endl;
    start = omp_get_wtime();
    
    mergeSort(a, 0, size-1);
    
    printArray(a, size);
    
    end = omp_get_wtime();
    
    cout<<"Time parallel = "<<(end-start)<<endl;
    
    return 0;
}

"""

In [9]:
text_file = open("merge11.cpp", "w")
text_file.write(code)
text_file.close()

In [10]:
!g++ -fopenmp merge11.cpp


In [11]:
!./a.out


  83  86  77  15  93  35  86  92  49  21  62  27  90  59  63  26  40  26  72  36  11  68  67  29  82  30  62  23  67  35  29  2  22  58  69  67  93  56  11  42  29  73  21  19  84  37  98  24  15  70  13  26  91  80  56  73  62  70  96  81  5  25  84  27  36  5  46  29  13  57  24  95  82  45  14  67  34  64  43  50  87  8  76  78  88  84  3  51  54  99  32  60  76  68  39  12  26  86  94  39
2  3  5  5  8  11  11  12  13  13  14  15  15  19  21  21  22  23  24  24  25  26  26  26  26  27  27  29  29  29  29  30  32  34  35  35  36  36  37  39  39  40  42  43  45  46  49  50  51  54  56  56  57  58  59  60  62  62  62  63  64  67  67  67  67  68  68  69  70  70  72  73  73  76  76  77  78  80  81  82  82  83  84  84  84  86  86  86  87  88  90  91  92  93  93  94  95  96  98  99  
Time parallel = 0.000226074
